In [1]:
#Tässä notebookissa analysoidaan tiedosto, jossa on lueteltu henkilön kuukausipalkka ja ikä. Data on tekoälyn
#tuottamaa. Analysoidaan palkkadata ja lopuksi tutkitaan, että korreloiko palkansaajan ikä palkan suuruuden kanssa.  

#ladataan aluksi data Excelistä tibleen.

library(readxl)

data <- read_excel("C:\\Users\\Installer\\Desktop\\palkatjaika.xlsx", sheet = "data")

print(data)

# A tibble: 2,000 × 2
     Ika Kuukausipalkka
   <dbl>          <dbl>
 1    49           4929
 2    42           3759
 3    51           8348
 4    63          11400
 5    45           4712
 6    46           2813
 7    49           5728
 8    45           3331
 9    44           4747
10    47           4354
# ℹ 1,990 more rows


In [2]:
#Määritetään seuraavaksi palkkojen tunnuslukuja komennolla skim, joka löytyy kirjastosta skimr.

library(skimr)

as.data.frame(skim(data$Kuukausipalkka)) 

#Jupyter vaatii, että skimin tulos muutetaan dataframeksi. R-studiossa voit kokeilla pelkkää skim-komentoa. 





skim_type,skim_variable,n_missing,complete_rate,numeric.mean,numeric.sd,numeric.p0,numeric.p25,numeric.p50,numeric.p75,numeric.p100,numeric.hist
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
numeric,data,0,1,4083.834,2239.763,1530,2514.75,3500,4749.75,12000,▇▅▁▁▁


In [3]:
#Skim ei määritä mediaania. Tulostetaan seuraavaksi tunnusluvut "perinteisellä" tavalla. 
library(dplyr)

data %>%
  summarise(
    keskiarvo = mean(Kuukausipalkka),
    mediaani = median(Kuukausipalkka),
    keskihajonta = sd(Kuukausipalkka),
    minimi  = min(Kuukausipalkka),
    maksimi  = max(Kuukausipalkka)
    
  )

#Jokaisen voisi laskea myös erikseen, mutta putkifunktion ja summarisen käyttö selventää. 


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




keskiarvo,mediaani,keskihajonta,minimi,maksimi
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4083.834,3500,2239.763,1530,12000


In [4]:
#Määritetään seuraavaksi desiilit. 

quantile(data$Kuukausipalkka,
         probs = seq(0, 1, 0.1),
         na.rm = TRUE)

#na.rm = TRUE poistaa NA:t (not available) laskennasta. 


0%     10%     20%     30%     40%     50%     60%     70%     80%     90% 
 1530.0  1923.8  2317.6  2712.4  3106.2  3500.0  4000.2  4499.9  5000.8  7001.4 
   100% 
12000.0

In [5]:
#Tehdään seuraavaksi frekvenssitaulukko

luokat <- seq(1000, 12000, by = 1000)

taulukko <- table(cut(data$Kuukausipalkka,
                   breaks = luokat,
                   include.lowest = TRUE,
                   right = TRUE))

taulukko



    [1e+03,2e+03]     (2e+03,3e+03]     (3e+03,4e+03]     (4e+03,5e+03] 
              239               508               453               400 
    (5e+03,6e+03]     (6e+03,7e+03]     (7e+03,8e+03]     (8e+03,9e+03] 
              100               100                40                40 
    (9e+03,1e+04]   (1e+04,1.1e+04] (1.1e+04,1.2e+04] 
               40                40                40 

In [6]:
#R käyttää oletuksena tieteellistä notaatiota. Asetetaan luokkarajojen otsikot itse. 
#Määritetään samalla luokkarajat siten, että viimeinen luokka on 11000-12000.

#Luokkarajat laskentaan
breaks <- c(seq(1000, 10000, by = 1000), 11000, 12001) 

#Luokkien alarajat otsikkoa varten
alar_ots <- seq(1000, 11000, by = 1000)

#Luokkien ylärajat otsikkoa varten. 
ylar_ots <- c(seq(1999, 10999, by = 1000), 12000)

#otsikot cut-funktiota varten

ots <- paste0(alar_ots, " - ", ylar_ots)


taulukko <- table(
  cut(
    data$Kuukausipalkka,
    breaks = breaks,
    labels = lab,
    right = FALSE,   #cutilla tehty jako ei sisällä ylintä arvoa: esim. (1000,2000[ --> 2000 ei kuulu tähän joukkoon. 
    include.lowest = TRUE #varmistetaan, että cutilla tehdyn jaon pienin arvo kuuluu mukaan. 
  )
)

taulukko



ERROR: Error: object 'lab' not found


In [ ]:
#Käytetään hyväksi aikaisemmin tehtyä ja tulostetaan frekvenssitaulukko tibleksi. 

library(dplyr)

frekv_taul <- tibble(
  luokka = names(taulukko),
  frekvenssi = as.numeric(taulukko)
) %>%
  
#mutate luo olemassa olevaan dataan uuden sarakkeen
  mutate(
    'suhteellinen_frekvenssi (%)' = 100*(frekvenssi / sum(frekvenssi)) 
      #Mikäli halutaan erikoismerkkejä, pitää sarakkeen nimi antaa heittomerkkien sisällä.
      #Ympäristöstä riippuen myös lainausmerkit saattavat toimia. 
  )


frekv_taul


In [ ]:
#Lisätään tähän vielä kumulatiivinen frekvenssi ja kumulatiivinen suhteellinen frekvenssi. 

frekv_taul <- frekv_taul %>%
  mutate(
    #Kumulatiivinen frekvenssi
    kumulatiivinen_f = cumsum(frekvenssi),   
    #Kumulatiivinen suhteellinen frekvenssi. Tässä on pakko käyttää ns. backtickejä (`), kun viitataan sarakkeen nimeen.
    'kumulatiivinen_suh (%)' = cumsum(`suhteellinen_frekvenssi (%)`)  
        
  )

frekv_taul

In [ ]:
#Lasketaan lopuksi Pearsonin korrelaatiokertoimen avulla, että korreloivatko ikä ja kuukausipalkka. 
#Tehdään hypoteesit ennen testiä: H0: Ikä ja palkka eivät korreloi. H1: Ikä ja palkka korreloivat. 
#Käytetään 95 % merkitsevyystasoa. 

cor.test(data$Ika, data$Kuukausipalkka, method = "pearson")

In [ ]:
#Nyt on 0,00000 % virhemahdollisuus, jos hylkäämme nollahypoteesin. Koska 95 % merkitsevyystasolla hyväksytään
#5 % virhemahdollisuus, nollahypoteesi hylätään vain vaihtoehtoinen hypoteesi jää voimaan -->
#ikä ja palkka siis korreloivat. 

#Lasketaan lopuksi korrelaatiokertoimen selitysaste. $estimate:lla saadaan pelkkä korrelaatiokertoimen arvo. 

(cor.test(data$Ika, data$Kuukausipalkka)$estimate)^2


In [ ]:
#Selitysaste on siis 48.7 %. Tämä tarkoittaa sitä, että 48,7 % palkan vaihtelusta voidaan
#selittää iän vaihtelulla. 